# Modelo de Risco de Crédito

### Extraição de dados usando Kaggle CLI


In [ ]:
%%bash
mkdir -p data && \
cd data && \ 
kaggle competitions download -c home-credit-credit-risk-model-stability && \
unzip home-credit-credit-risk-model-stability.zip && \
rm -rf parquet_files \